In [1]:
import ee
import geemap

/Users/bryan/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


todos:

* figure out centering and clipping to coords of interest
* plotting multiple layers
* saving images
* plotting time-series data
    * grabbing snapshots from a certain range of time


In [3]:
ee.Authenticate()
# ee.Initialize(project='project-17402')
ee.Initialize(project='fiery-glass-441301-e5')

In [23]:
dataset_name = test_dataset = "CSIC/SPEI/2_9"
images = ee.ImageCollection(dataset_name).mosaic()
# info = images.limit(5).getInfo()

# For now, visualize the first image only
# image = images.first()
image = images

dataset_name_no_slashes = dataset_name.replace("/", "_")
image_path = f"tmp/{dataset_name_no_slashes}.tif"
# TODO: make thi not just mongolia lol
roi = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')\
    .filter(ee.Filter.eq('country_na', 'Mongolia'))\
    .geometry()
# geemap.ee_exp
# Set the visualization ranges and color palette
# vis_params = {
#     'min': -2.33,
#     'max': 2.33,
#     'palette': [
#         '8b1a1a', 'de2929', 'f3641d',
#         'fdc404', '9afa94', '03f2fd',
#         '12adf3', '1771de', '00008b',
#     ]
# }
# band_viz = {
#   "min": 0.0,
#   "max": 0.00005,
#   "opacity": 1.0,
#   "palette": ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
# }
# Create the map object centered on Spain
map_object = geemap.Map()
# map_object.setCenter(-3.75, 40.47, 4)
map_object.centerObject((roi), 4)

# Add the SPEI 24-month layer to the map
map_object.addLayer(image.clip(roi), vis_params, 'SPEI')
display(map_object)
# map_object.addLayer(famine.clip(roi), band_viz, 'evap_tavg')

# map_object.addLayer(spei24, vis_params, 'SPEI 24 month')
# map_object.addLayer(lai, {}, 'LAI')

Map(center=[46.9657716979408, 103.14085456837168], controls=(WidgetControl(options=['position', 'transparent_b…

let's try case 1 evapotranspiration

In [6]:
# Retrieve the last date from the SPEI dataset
drought_dataset = ee.ImageCollection("CSIC/SPEI/2_9").filterDate('2022-12-01', '2023-01-01')

# Select the 24-month analysis
spei = drought_dataset.select('SPEI_24_month').mosaic()


dataset = ee.ImageCollection('NASA/FLDAS/NOAH01/C/GL/M/V001').filter(ee.Filter.date('2018-11-01', '2018-12-01'))
famine = dataset.select('Evap_tavg').mosaic()

plant_cover_dataset = ee.ImageCollection("NOAA/CDR/AVHRR/LAI_FAPAR/V5")
lai = plant_cover_dataset.select('LAI').mosaic()


In [13]:
sahel_region = ee.Geometry.Rectangle([
    -17.0,  # West longitude (Senegal)
    12.0,   # South latitude
    35.0,   # East longitude (Sudan)
    18.0    # North latitude
])

# maybe the mongolia datasets will be easier

# Get Mongolia's boundary from Earth Engine's country feature collection
mongolia = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')\
    .filter(ee.Filter.eq('country_na', 'Mongolia'))\
    .geometry()


In [14]:
# Set the visualization ranges and color palette
vis_params = {
    # 'min': -2.33,
    # 'max': 2.33,
    # 'palette': [
    #     '8b1a1a', 'de2929', 'f3641d',
    #     'fdc404', '9afa94', '03f2fd',
    #     '12adf3', '1771de', '00008b',
    # ]
}
band_viz = {
  # "min": 0.0,
  # "max": 0.00005,
  # "opacity": 1.0,
  # "palette": ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}
# Create the map object centered on Spain
map_object = geemap.Map()
# map_object.setCenter(-3.75, 40.47, 4)
map_object.centerObject((sahel_region), 4)

# Add the SPEI 24-month layer to the map
map_object.addLayer(spei.clip(sahel_region), vis_params, 'SPEI')
map_object.addLayer(famine.clip(sahel_region), band_viz, 'evap_tavg')

# map_object.addLayer(spei24, vis_params, 'SPEI 24 month')
# map_object.addLayer(lai, {}, 'LAI')

In [15]:
geemap.ee_export_image(map_object, filename='map_object.jpg', scale=1000, region=sahel_region, file_per_band=False)

The ee_object must be an ee.Image.


In [16]:
display(map_object)


Map(center=[16.575995991319147, 8.999999999999998], controls=(WidgetControl(options=['position', 'transparent_…

In [78]:
# Define visualization parameters
leaf_area_index_vis = {
    'min': 0.0,
    'max': 4000.0,
    'palette': ['3b0200', '977705', 'ca9f06', 'ffca09', '006a03', '003b02']
}

protected_areas_viz = {
  "palette": ['2ed033', '5aff05', '67b9ff', '5844ff', '0a7618', '2c05ff'],
  "min": 0.0,
  "max": 1550000.0,
  "opacity": 0.8,
}
# Create the map object and set center
map_object2 = geemap.Map()
map_object2.setCenter(20, 24.5, 2)

# # Add the LAI layer to the map
# map_object2.addLayer(lai, leaf_area_index_vis, 'Leaf Area Index')

# Define Sahel region boundaries (approximate)
sahel_bounds = ee.Geometry.Rectangle([
    -17.0,  # West longitude (Senegal)
    12.0,   # South latitude
    35.0,   # East longitude (Sudan)
    18.0    # North latitude
])

# Set the map view to the Sahel region
map_object2.centerObject(sahel_bounds, 4)

# Add the LAI layer to the map
map_object2.addLayer(protected_areas, None, 'Leaf Area Index')

# Add the Sahel region rectangle to the map for reference

# # Create a circle by drawing a 20000 meter buffer around a point.
# roi = ee.Geometry.Point([-122.4481, 37.7599]).buffer(20000)
# lai_clipped = lai.clip(roi)

# # Add the image layer to the map and display it.
# map_object2.add_layer(lai_clipped, leaf_area_index_vis, 'mosaic clipped')
display(map_object2)

Map(center=[16.575995991319147, 8.999999999999998], controls=(WidgetControl(options=['position', 'transparent_…

In [29]:
geemap.ee_export_image(map_object2, "test.png", scale=None, crs=None, region=None, file_per_band=False)

The ee_object must be an ee.Image.


In [70]:
drained_soil_dataset = ee.ImageCollection("FAO/GHG/1/DROSA_A").mosaic()
soil_organic_carbon = ee.Image('OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02')
protected_areas = ee.FeatureCollection("WCMC/WDPA/current/polygons")

In [ ]:
# Set the visualization ranges and color palette
vis_params = {
    'min': -2.33,
    'max': 2.33,
    'palette': [
        '8b1a1a', 'de2929', 'f3641d',
        'fdc404', '9afa94', '03f2fd',
        '12adf3', '1771de', '00008b',
    ]
}
band_viz = {
  "min": 0.0,
  "max": 0.00005,
  "opacity": 1.0,
  "palette": ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}

drained_soil_viz = {
  "bands": ['cropland'],
  "min": 1.0,
  "max": 60.0,
  "palette": ['white', 'red']
}

soil_carbon_viz = {
  "bands": ['b0'],
  "min": 0.0,
  "max": 120.0,
  "palette": [
    'ffffa0','f7fcb9','d9f0a3','addd8e','78c679','41ab5d',
    '238443','005b29','004b29','012b13','00120b',
  ]
}
protected_areas_viz = {
  "palette": ['2ed033', '5aff05', '67b9ff', '5844ff', '0a7618', '2c05ff'],
  "min": 0.0,
  "max": 1550000.0,
  "opacity": 0.8,
}

# Create the map object centered on Spain
map_object3 = geemap.Map()
# map_object.setCenter(-3.75, 40.47, 4)
map_object3.centerObject((mongolia), 4)

# Add the SPEI 24-month layer to the map
map_object3.addLayer(spei.clip(mongolia), vis_params, 'SPEI')
map_object3.addLayer(famine.clip(mongolia), band_viz, 'evap_tavg')
map_object3.addLayer(lai.clip(mongolia), leaf_area_index_vis, 'leaf area index')
map_object3.addLayer(drained_soil_dataset.clip(mongolia), drained_soil_viz, 'drained soil')
map_object3.addLayer(soil_organic_carbon.clip(mongolia), soil_carbon_viz, 'soil carbon')
map_object3.addLayer(protected_areas, protected_areas_viz, 'protected areas')

# map_object.addLayer(spei24, vis_params, 'SPEI 24 month')
# map_object.addLayer(lai, {}, 'LAI')

EEException: Image.visualize: Cannot provide a palette when visualizing more than one band.

In [ ]:
display(map_object3)

Map(center=[46.9657716979408, 103.14085456837168], controls=(WidgetControl(options=['position', 'transparent_b…

In [76]:
dataset = ee.FeatureCollection('WCMC/WDPA/current/polygons')
visParams = {
  "palette": ['2ed033', '5aff05', '67b9ff', '5844ff', '0a7618', '2c05ff'],
  "min": 0.0,
  "max": 1550000.0,
  "opacity": 0.8,
}
image = ee.Image().float().paint(dataset, 'REP_AREA')



# geemap.Map.setCenter(41.104, -17.724, 6)
geemap.Map.addLayer(image, visParams, 'WCMC/WDPA/current/polygons')
geemap.Map.addLayer(dataset, null, 'for Inspector', false)

AttributeError: 'Image' object has no attribute 'ee_layers'

In [87]:
!pip install mss

In [ ]:
# map_object3.to_image(filename="/Users/davianho/Desktop/test.png", monitor=1)

In [94]:
map_object.to_image('/Users/davianho/Downloads/mongolia_lai_map.png')


In [96]:
map_object.save('sahel.html')

In [97]:
map_object3.save('mongolia_lai_map.html')